In [8]:
#spark.stop()  # Stop the existing session

In [9]:
#!pip install pyspark
#!pip install matplotlib
#!pip install pandas
#!pip install plotly
#!pip install delta-spark==3.1.0

In [69]:
import pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import desc
from pyspark.sql.functions import count
from delta import *
import logging

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [11]:
archivo01 = "/home/dataset/SECOP_II_-_Procesos_de_Contrataci_n_20240320.csv"

In [12]:
data01 = spark.read.csv(archivo01, header=True, inferSchema=True)

In [13]:
data01.count()

6434461

In [14]:
data01.printSchema()

root
 |-- Entidad: string (nullable = true)
 |-- Nit Entidad: string (nullable = true)
 |-- Departamento Entidad: string (nullable = true)
 |-- Ciudad Entidad: string (nullable = true)
 |-- OrdenEntidad: string (nullable = true)
 |-- Entidad Centralizada: string (nullable = true)
 |-- ID del Proceso: string (nullable = true)
 |-- Referencia del Proceso: string (nullable = true)
 |-- PCI: string (nullable = true)
 |-- ID del Portafolio: string (nullable = true)
 |-- Nombre del Procedimiento: string (nullable = true)
 |-- Descripción del Procedimiento: string (nullable = true)
 |-- Fase: string (nullable = true)
 |-- Fecha de Publicacion del Proceso: string (nullable = true)
 |-- Fecha de Ultima Publicación: string (nullable = true)
 |-- Fecha de Publicacion (Fase Planeacion Precalificacion): string (nullable = true)
 |-- Fecha de Publicacion (Fase Seleccion Precalificacion): string (nullable = true)
 |-- Fecha de Publicacion (Manifestacion de Interes): string (nullable = true)
 |-- Fech

In [63]:
try: 
    data02 = data01 \
        .withColumnRenamed("Nit Entidad", "NitEntidad") \
        .withColumnRenamed("Departamento Entidad", "DepartamentoEntidad") \
        .withColumnRenamed("Ciudad Entidad", "CiudadEntidad") \
        .withColumnRenamed("OrdenEntidad", "OrdenEntidad") \
        .withColumnRenamed("Entidad Centralizada", "EntidadCentralizada") \
        .withColumnRenamed("ID del Proceso", "IDdelProceso") \
        .withColumnRenamed("Referencia del Proceso", "ReferenciadelProceso") \
        .withColumnRenamed("PCI", "PCI") \
        .withColumnRenamed("ID del Portafolio", "IDdelPortafolio") \
        .withColumnRenamed("Nombre del Procedimiento", "NombredelProcedimiento") \
        .withColumnRenamed("Descripción del Procedimiento", "DescripciondelProcedimiento") \
        .withColumnRenamed("Fase", "Fase") \
        .withColumnRenamed("Fecha de Publicacion del Proceso", "FechadePublicaciondelProceso") \
        .withColumnRenamed("Fecha de Ultima Publicación", "FechadeUltimaPublicacion") \
        .withColumnRenamed("Fecha de Publicacion (Fase Planeacion Precalificacion)", "FechadePubFasePlanPreca") \
        .withColumnRenamed("Fecha de Publicacion (Fase Seleccion Precalificacion)", "FechadePubFaseSelePreca") \
        .withColumnRenamed("Fecha de Publicacion (Manifestacion de Interes)", "FechadePubManifdeInt") \
        .withColumnRenamed("Fecha de Publicacion (Fase Borrador)", "FechadePublicacionFaseBorrador") \
        .withColumnRenamed("Fecha de Publicacion (Fase Seleccion)", "FechadePublicacionFaseSeleccion") \
        .withColumnRenamed("Precio Base", "PrecioBase") \
        .withColumnRenamed("Modalidad de Contratacion", "ModalidaddeContratacion") \
        .withColumnRenamed("Justificación Modalidad de Contratación", "JustificacionModalidaddeContratacion") \
        .withColumnRenamed("Duracion", "Duracion") \
        .withColumnRenamed("Unidad de Duracion", "UnidaddeDuracion") \
        .withColumnRenamed("Fecha de Recepcion de Respuestas", "FechadeRecepciondeRespuestas") \
        .withColumnRenamed("Fecha de Apertura de Respuesta", "FechadeAperturadeRespuesta") \
        .withColumnRenamed("Fecha de Apertura Efectiva", "FechadeAperturaEfectiva") \
        .withColumnRenamed("Ciudad de la Unidad de Contratación", "CiudaddelaUnidaddeContratacion") \
        .withColumnRenamed("Nombre de la Unidad de Contratación", "NombredelaUnidaddeContratacion") \
        .withColumnRenamed("Proveedores Invitados", "ProveedoresInvitados") \
        .withColumnRenamed("Proveedores con Invitacion Directa", "ProveedoresconInvitacionDirecta") \
        .withColumnRenamed("Visualizaciones del Procedimiento", "VisualizacionesdelProcedimiento") \
        .withColumnRenamed("Proveedores que Manifestaron Interes", "ProveedoresqueManifestaronInteres") \
        .withColumnRenamed("Respuestas al Procedimiento", "RespuestasalProcedimiento") \
        .withColumnRenamed("Respuestas Externas", "RespuestasExternas") \
        .withColumnRenamed("Conteo de Respuestas a Ofertas", "ConteodeRespuestasaOfertas") \
        .withColumnRenamed("Proveedores Unicos con Respuestas", "ProveedoresUnicosconRespuestas") \
        .withColumnRenamed("Numero de Lotes", "NumerodeLotes") \
        .withColumnRenamed("Estado del Procedimiento", "EstadodelProcedimiento") \
        .withColumnRenamed("ID Estado del Procedimiento", "IDEstadodelProcedimiento") \
        .withColumnRenamed("Adjudicado", "Adjudicado") \
        .withColumnRenamed("ID Adjudicacion", "IDAdjudicacion") \
        .withColumnRenamed("CodigoProveedor", "CodigoProveedor") \
        .withColumnRenamed("Departamento Proveedor", "DepartamentoProveedor") \
        .withColumnRenamed("Ciudad Proveedor", "CiudadProveedor") \
        .withColumnRenamed("Fecha Adjudicacion", "FechaAdjudicacion") \
        .withColumnRenamed("Valor Total Adjudicacion", "ValorTotalAdjudicacion") \
        .withColumnRenamed("Nombre del Adjudicador", "NombredelAdjudicador") \
        .withColumnRenamed("Nombre del Proveedor Adjudicado", "NombredelProveedorAdjudicado") \
        .withColumnRenamed("NIT del Proveedor Adjudicado", "NITdelProveedorAdjudicado") \
        .withColumnRenamed("Codigo Principal de Categoria", "CodigoPrincipaldeCategoria") \
        .withColumnRenamed("Estado de Apertura del Proceso", "EstadodeAperturadelProceso") \
        .withColumnRenamed("Tipo de Contrato", "TipodeContrato") \
        .withColumnRenamed("Subtipo de Contrato", "SubtipodeContrato") \
        .withColumnRenamed("Categorias Adicionales", "CategoriasAdicionales") \
        .withColumnRenamed("URLProceso", "URLProceso") \
        .withColumnRenamed("Codigo Entidad", "CodigoEntidad") \
        .withColumnRenamed("Estado Resumen", "EstadoResumen")
except Exception as e:
    logging.error("Error al renombrar columnas: {}".format(str(e)))
    raise

In [64]:
#columnas_renombradas = {"Nit Entidad": "Nit_Entidad", "Departamento Entidad": "Departamento_Entidad"}

#data02 = data01
#for old_name, new_name in columnas_renombradas.items():
#    data02 = data02.withColumnRenamed(old_name, new_name)

#data02.show()  # Optional: View the renamed DataFrame

In [65]:
data02.printSchema()

root
 |-- Entidad: string (nullable = true)
 |-- NitEntidad: string (nullable = true)
 |-- DepartamentoEntidad: string (nullable = true)
 |-- CiudadEntidad: string (nullable = true)
 |-- OrdenEntidad: string (nullable = true)
 |-- EntidadCentralizada: string (nullable = true)
 |-- IDdelProceso: string (nullable = true)
 |-- ReferenciadelProceso: string (nullable = true)
 |-- PCI: string (nullable = true)
 |-- IDdelPortafolio: string (nullable = true)
 |-- NombredelProcedimiento: string (nullable = true)
 |-- DescripciondelProcedimiento: string (nullable = true)
 |-- Fase: string (nullable = true)
 |-- FechadePublicaciondelProceso: string (nullable = true)
 |-- FechadeUltimaPublicacion: string (nullable = true)
 |-- FechadePubFasePlanPreca: string (nullable = true)
 |-- FechadePubFaseSelePreca: string (nullable = true)
 |-- FechadePubManifdeInt: string (nullable = true)
 |-- FechadePublicacionFaseBorrador: string (nullable = true)
 |-- FechadePublicacionFaseSeleccion: string (nullable =

In [70]:
ruta01deltalake01 = "/home/app/datalake/1bronze/SECOPIIProcesosContratacion"

In [71]:
try:
    data02.write.format("delta").mode("overwrite").save(ruta01deltalake01)
    logging.info("Datos guardados correctamente en Delta Lake.")
except Exception as e:
    logging.error("Error al guardar datos en Delta Lake: {}".format(str(e)))
    raise

In [68]:
# Escribe el DataFrame en Delta Lake
#data01.write.format("delta").mode("overwrite").save(ruta01deltalake01)

In [72]:
data03 = spark.read.format("delta").load(ruta01deltalake01)

In [73]:
data03.count()

6434461

In [74]:
data03.printSchema()

root
 |-- Entidad: string (nullable = true)
 |-- NitEntidad: string (nullable = true)
 |-- DepartamentoEntidad: string (nullable = true)
 |-- CiudadEntidad: string (nullable = true)
 |-- OrdenEntidad: string (nullable = true)
 |-- EntidadCentralizada: string (nullable = true)
 |-- IDdelProceso: string (nullable = true)
 |-- ReferenciadelProceso: string (nullable = true)
 |-- PCI: string (nullable = true)
 |-- IDdelPortafolio: string (nullable = true)
 |-- NombredelProcedimiento: string (nullable = true)
 |-- DescripciondelProcedimiento: string (nullable = true)
 |-- Fase: string (nullable = true)
 |-- FechadePublicaciondelProceso: string (nullable = true)
 |-- FechadeUltimaPublicacion: string (nullable = true)
 |-- FechadePubFasePlanPreca: string (nullable = true)
 |-- FechadePubFaseSelePreca: string (nullable = true)
 |-- FechadePubManifdeInt: string (nullable = true)
 |-- FechadePublicacionFaseBorrador: string (nullable = true)
 |-- FechadePublicacionFaseSeleccion: string (nullable =

In [78]:
# Obtener el historial de versiones
delta_table = DeltaTable.forPath(spark, ruta01deltalake01)
version_history = delta_table.history()

# Mostrar el historial de versiones
version_history.show()

+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      0|2024-03-22 22:50:...|  NULL|    NULL|    WRITE|{mode -> Overwrit...|NULL|    NULL|     NULL|       NULL|  Serializable|        false|{numFiles -> 42, ...|        NULL|Apache-Spark/3.5....|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+



In [76]:
data01agrupar01 = data03.groupBy("TipodeContrato")
data01cantidad01 = data01agrupar01.count()
data01cantidad01 = data01cantidad01.orderBy(desc("count"))

In [77]:
data01cantidad01.show(truncate=False)

+--------------------------+-------+
|TipodeContrato            |count  |
+--------------------------+-------+
|Prestación de servicios   |3078131|
|NULL                      |1514017|
|Decreto 092 de 2017       |519652 |
|Suministros               |403405 |
|Otro                      |233554 |
|Compraventa               |230974 |
|Acuerdo Marco de Precios  |159927 |
|No                        |91964  |
|Obra                      |82271  |
|Arrendamiento de inmuebles|30273  |
|Interventoría             |20095  |
|Consultoría               |16725  |
|Seguros                   |16706  |
|No Especificado           |12479  |
|Comodato                  |8959   |
|Arrendamiento de muebles  |4545   |
|Venta muebles             |2831   |
|Servicios financieros     |2512   |
|Concesión                 |1297   |
|Asociación Público Privada|1135   |
+--------------------------+-------+
only showing top 20 rows

